Install dependencies

In [ ]:
! pip install pandas;
! pip install pgmpy;

Import dependencies

In [27]:
import pandas as pd;
from pgmpy.models import BayesianNetwork

Initialise Data

In [28]:
Quali = "Qualifikation"
S = "Schnitt"
BL = "Bundesland"
M = "Mathe"
P = "Physik"
D = "Deutsch"
ST = "Schultyp"
OLTM = "OLT-Mathe"
OLTD = "OLT-Deutsch"
SFT = "Studierfähigkeitstest"
A = "Alter"
Geschl = "Geschlecht"
JEDE = "Jahreseinkommen der Eltern"
SB = "Staatsbürgerschaft"
SG = "Studiengang"
Absch= "Abschluss"

Import data from csv File

In [29]:
init_data = pd.read_csv('p001_1.csv', sep=';',dtype={
    Quali: 'string',
    S: 'string',
    BL: 'string',
    M: 'string',
    D: 'string',
    P: 'string',
    ST: "string",
    OLTM: "string",
    OLTD: "string",
    SFT: "string",
    A: "string",
    Geschl: "string",
    JEDE: "string",
    SB: "string",
    SG: "string",
    Absch: "string"
})
init_data = init_data.replace({'n.a.': "-1"})
init_data = init_data.replace({ "keine": "-1"})
init_data[S] = init_data[S].str.replace(",",".").astype('float') 
init_data[M] = init_data[M].str.replace(",",".").astype('float') 
init_data[D] = init_data[D].str.replace(",",".").astype('float') 
init_data[P] = init_data[P].str.replace(",",".").astype('float') 

init_data[OLTM] = init_data[OLTM].astype('int') 
init_data[OLTD] = init_data[OLTD].astype('int') 
init_data[SFT] = init_data[SFT].astype('int') 
init_data[A] = init_data[A].astype('int') 
init_data[JEDE] = init_data[JEDE].astype('int') 
init_data = init_data.replace({"-1": None})
init_data = init_data.replace({-1: None})
init_data = init_data.replace({-1.0: None})
display(init_data)

,Qualifikation,Schnitt,Bundesland,Mathe,Physik,Deutsch,Schultyp,OLT-Mathe,OLT-Deutsch,Studierfähigkeitstest,Alter,Geschlecht,Jahreseinkommen der Eltern,Staatsbürgerschaft,Studiengang,Abschluss
0,Abitur,2.7,Baden-Württemberg,2.3,2.2,2.1,Allgemeinbildendes Gymnasium,63,62,685,19,m,47000,deutsch,Maschinenbau,3
1,Meister,1.6,Nordrhein-Westfalen,None,None,None,None,36,59,None,25,m,87000,deutsch,Maschinenbau,abgebrochen
2,Abitur,1.8,Baden-Württemberg,1.0,1.1,1.2,Wirtschaftsgymnasium,96,94,None,18,w,115000,deutsch,Soziale Arbeit,"1,1"
3,Abitur,1.1,Baden-Württemberg,2.5,2.7,1.9,Technisches Gymnasium,70,76,None,17,m,115000,deutsch,Elektrotechnik,"2,6"
4,Abitur,1.4,Bayern,2.0,2.1,1.4,Wirtschaftsgymnasium,65,82,None,17,m,90000,deutsch,Elektrotechnik,"2,3"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Abitur,1.4,Baden-Württemberg,2.4,2.5,2.5,Allgemeinbildendes Gymnasium,60,65,None,19,w,107000,deutsch,Elektrotechnik,"3,1"
96,Abitur,2.6,Baden-Württemberg,1.1,1.1,1.2,Technisches Gymnasium,100,94,None,20,m,80000,deutsch,Wirtschaftswissenschaften,"1,5"
97,Abitur,1.6,Baden-Württemberg,2.0,2.2,1.7,Technisches Gymnasium,76,86,None,19,m,208000,deutsch,Maschinenbau,"2,4"
98,Abitur,2.6,Bayern,1.5,1.4,1.1,Allgemeinbildendes Gymnasium,80,86,None,18,m,110000,deutsch,Maschinenbau,"1,8"


Create Bayesian Network with Nodes and Edges and init_data

In [30]:


G = BayesianNetwork()
G.add_nodes_from(init_data.columns)

G.add_edges_from([(ST, SG),(Quali, S),(S, M),(S,D),(S,P),(P, OLTM),(M, OLTM),(D, OLTD),(OLTM, Absch),(OLTD, Absch),(BL, JEDE),(JEDE, SFT),(JEDE, SG),(SG,Absch),(A, SFT),(Geschl, SFT),(SB, SFT)])
G.fit(init_data)


Simulate Data with input_data

In [31]:

input_data = {
    BL:"Baden-Württemberg", 
    ST:"Allgemeinbildendes Gymnasium"
    }
simulate = G.simulate(n_samples=1, do=input_data)
display(simulate)

  0%|          | 0/1 [00:00<?, ?it/s]

,Schnitt,Mathe,Qualifikation,Bundesland,Geschlecht,Studierfähigkeitstest,Schultyp,Alter,Physik,OLT-Deutsch,Studiengang,Jahreseinkommen der Eltern,Abschluss,Staatsbürgerschaft,Deutsch,OLT-Mathe
0,3.4,1.8,Techniker,Baden-Württemberg,w,784,Allgemeinbildendes Gymnasium,18,2.2,73,Elektrotechnik,79000,2,EU Bürger,1.3,76


Predict Data with input_data

In [32]:

input_data = pd.DataFrame({
    Quali: ["Abitur"],
    S: [2.7],
    BL: ["Baden-Württemberg"],
    M: [2.3],
    P: [2.2],
    D: [2.1],
    ST: ["Allgemeinbildendes Gymnasium"],
    OLTM: [63],
    OLTD: [62],
    SFT: [685],
    A: ["19"],
    Geschl: ["m"],
    JEDE: [47000],
    SB: ["deutsch"],
    SG: ["Maschinenbau"],
    })
predict = G.predict(input_data)
display(predict)

  0%|          | 0/1 [00:00<?, ?it/s]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices